# Exploratory notebook to inspect source data and validate transformation logic used in the ETL pipeline.


## Imports & context
Purpose: make the notebook runnable and clear.

In [74]:
from pathlib import Path
import pandas as pd
from pandas.api.types import is_numeric_dtype


## Load the source table
Purpose: verify that the correct table is extracted.

In [75]:
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"

tables = pd.read_html(url, match="Market cap")
len(tables)

1

## Inspect raw structure
Purpose: understand column names and raw values.

In [76]:
df_raw = tables[0]
df_raw.head()

,Rank,Bank name,Market cap (US$ billion)
0,1,JPMorgan Chase,432.92
1,2,Bank of America,231.52
2,3,Industrial and Commercial Bank of China,194.56
3,4,Agricultural Bank of China,160.68
4,5,HDFC Bank,157.91


## Clean and normalize core columns
Purpose: validate the logic that will go into banks_etl.py.

In [77]:
# Rename columns
df_clean = df_raw.copy()
cols = df_clean.columns.tolist()
df_clean = df_clean.rename(
    columns={
        cols[1]: "Name",
        cols[2]: "MC_USD_Billion"
    }
)
df_clean.head()

,Rank,Name,MC_USD_Billion
0,1,JPMorgan Chase,432.92
1,2,Bank of America,231.52
2,3,Industrial and Commercial Bank of China,194.56
3,4,Agricultural Bank of China,160.68
4,5,HDFC Bank,157.91


In [78]:
# Intentionally corrupt MC_USD_Billion values to simulate real-world dirty data:
df_clean["MC_USD_Billion"] = df_clean["MC_USD_Billion"].astype("object")
df_clean.loc[1, "MC_USD_Billion"] = "zero "
df_clean.loc[2, "MC_USD_Billion"] = "194.56 \n"
df_clean.loc[3, "MC_USD_Billion"] = "N/A"
df_clean.loc[4, "MC_USD_Billion"] = "157,91"
df_clean.loc[0, "Name"] = "N/A "
df_clean.head()

,Rank,Name,MC_USD_Billion
0,1,N/A,432.92
1,2,Bank of America,zero
2,3,Industrial and Commercial Bank of China,194.56 \n
3,4,Agricultural Bank of China,N/A
4,5,HDFC Bank,"157,91"


In [79]:
NA_MARKERS = ["N/A", "n/a", "NA", "", "null", "None"]

df_clean["Name"] = df_clean["Name"].str.strip().replace(NA_MARKERS, pd.NA)

if not is_numeric_dtype(df_clean["MC_USD_Billion"]):
    df_clean["MC_USD_Billion"] = (df_clean["MC_USD_Billion"]
                             .astype(str)
                             .str.strip() 
                             .str.replace(r"[,\n]", "", regex=True)
                             .str.replace(",", ".", regex=True)
                             .pipe(pd.to_numeric, errors="coerce"))
df_clean.head()

,Rank,Name,MC_USD_Billion
0,1,<NA>,432.92
1,2,Bank of America,NaN
2,3,Industrial and Commercial Bank of China,194.56
3,4,Agricultural Bank of China,NaN
4,5,HDFC Bank,15791.00


In [80]:
# Validate required columns and drop rows with missing critical values,
# reporting how many records were removed
check_cols = ["Name", "MC_USD_Billion"]
invalid_mask = df_clean[check_cols].isna().any(axis=1)
if invalid_mask.any():
    dropped = invalid_mask.sum()
    total = len(df_clean)
    print(f"Warning: dropping {dropped}/{total} invalid rows\n")
    print(df_clean.loc[invalid_mask])
    print("")
    clean_banks = df_clean.loc[~invalid_mask].reset_index(drop=True)
else:
    clean_banks = df_clean
clean_banks.head()


   Rank                        Name  MC_USD_Billion
0     1                        <NA>          432.92
1     2             Bank of America             NaN
3     4  Agricultural Bank of China             NaN



,Rank,Name,MC_USD_Billion
0,3,Industrial and Commercial Bank of China,194.56
1,5,HDFC Bank,15791.00
2,6,Wells Fargo,155.87
3,7,HSBC Holdings PLC,148.90
4,8,Morgan Stanley,140.83


## Currency enrichment test

In [81]:
RATES_URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv"
rates = pd.read_csv(RATES_URL)
rates.head()

,Currency,Rate
0,EUR,0.93
1,GBP,0.80
2,INR,82.95


In [82]:
rate_dict = dict(zip(rates["Currency"], rates["Rate"]))

for cur, rate in rate_dict.items():
    if cur == "USD":
        continue
    clean_banks[f"MC_{cur}_Billion"] = (clean_banks["MC_USD_Billion"]*rate).round(2)
clean_banks.head()    

,Rank,Name,MC_USD_Billion,MC_EUR_Billion,MC_GBP_Billion,MC_INR_Billion
0,3,Industrial and Commercial Bank of China,194.56,180.94,155.65,16138.75
1,5,HDFC Bank,15791.00,14685.63,12632.80,1309863.45
2,6,Wells Fargo,155.87,144.96,124.70,12929.42
3,7,HSBC Holdings PLC,148.90,138.48,119.12,12351.26
4,8,Morgan Stanley,140.83,130.97,112.66,11681.85
